# Katib 실습
katib을 사용하여 training code `01-covid19-katib-train.py`를 hyper parameter tunning한다

# 사전준비1. Base Image
Host에 ssh 접속하여 아래와 같이 docker build하고 push한다

```sh
cat << EOF > Dockerfile.kf-base
FROM ubuntu:18.04
ARG DEBIAN_FRONTEND=noninteractive

RUN apt-get update && \
    apt-get install -y python3 python3-pip python3-dev ffmpeg libsm6 libxext6 

RUN  pip3 install --upgrade pip && \
     pip install \
        tensorflow \
        sklearn \
        opencv-python \   
        pillow \
        minio
EOF
```

```sh
# docker build script
REGISTRY=kubeflow-registry.default.svc.cluster.local:30000
TAG=$REGISTRY/kf-base:latest

docker build -f Dockerfile.kf-base -t $TAG . && \
docker push $TAG
```

# 사전준비2. s3 repository(Minio)에 training data 적재
실습 코드 중 `00-uploadDataset-covid19.sh`을 사용
```sh
# github에서 training data 다운로드
git clone https://github.com/zhongli1990/Covid19-X-Rays.git

# Minio에 버킷 생성
mc mb myminio/dataset/covid19

# Minio에  training data 적재
mc cp -r Covid19-X-Rays/all/ myminio/dataset/covid19/
```

# Step 1. Hyperparamer 수행할 로직을 Containerizing

### 방법 1. docker 명령으로 build
**Host에 ssh 접속**하여 `01-covid19-katib-train.py`를 업로드한 후

사전준비에서 마련한 Base Image를 이용하여 아래와 같이 docker build하고 push한다

```sh
# docker build script
REGISTRY=kubeflow-registry.default.svc.cluster.local:30000
TAG=$REGISTRY/covid19-katib-job:latest

cat << EOF > Dockerfile.covid 
FROM $REGISTRY/kf-base:latest
COPY 01-covid19-katib-train.py /app/
CMD ['python3', '/app/01-covid19-katib-train.py']
EOF

docker build -f Dockerfile.covid -t $TAG . && \
docker push $TAG
```

### 방법 2. fairing으로 build
아래 셀을 실행하여 fairing으로 image build하면 image tag명이난수로 생성된다
(기존 fairing에서 build만 실행하기 실습과 동일한 코드이다)

In [2]:
from kubeflow.fairing.preprocessors.base import BasePreProcessor 
from kubeflow.fairing.builders.append.append import AppendBuilder

PRIVATE_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'

preproc = BasePreProcessor(
    input_files = ['01-covid19-katib-train.py'], 
    #path_prefix = '/myapp', # default: /app
    command = ['python3'],  # default: python
)    

builder = AppendBuilder(
    preprocessor = preproc,    
    base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', # 사전준비에서 마련한 Base Image
    image_name = 'covid19-katib-job',
    registry = PRIVATE_REGISTRY,
    push = True,
)

builder.build()

print(f'Published Docker image with tag: "{builder.image_tag}"')

[W 201207 01:47:34 append:50] Building image using Append builder...
[I 201207 01:47:34 base:107] Creating docker context: /tmp/fairing_context_0pexxmjj
[I 201207 01:47:34 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.cluster.local:30000/kf-base:latest'
[W 201207 01:47:34 append:54] Image successfully built in 0.037106996000147774s.
[W 201207 01:47:34 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/covid19-katib-job:ABC1BD46...
[I 201207 01:47:34 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.cluster.local:30000/covid19-katib-job:ABC1BD46'
[W 201207 01:47:34 append:81] Uploading kubeflow-registry.default.svc.cluster.local:30000/covid19-katib-job:ABC1BD46
[I 201207 01:47:34 docker_session_:280] Layer sha256:0d3160e1d0de4061b5b32ee09af687b898921d36ed9556df5910ddc3104449cd exists, skipping
[I 201207 01:47:34 docker_session_:280] Layer sha256:4d220205e6561101f16a16ded38672d

Published Docker image with tag: "kubeflow-registry.default.svc.cluster.local:30000/covid19-katib-job:ABC1BD46"


# Step 2. Katib 실행
※ 주의: faring으로 containerizing한 경우, fairing이 생성한 image tag명으로 `03-covid19-katib-random.yaml`내용을 수정한다

```yaml
apiVersion: "kubeflow.org/v1alpha3"
kind: Experiment
...
spec:
...
  trialTemplate:
    goTemplate:
...
          spec:
            template:
              spec:
                containers:
                - name: {{.Trial}}
                  image: kubeflow-registry.default.svc.cluster.local:30000/katib-job:latest  # 여기를 수정
```

### 방법 1. kubectl로 apply
Host에 ssh 접속하여 아래와 같이 kubectl apply 실행
```sh
kubectl apply -f 03-covid19-katib-random.yaml
```

### 방법 2. katib UI에서 submit
katib UI의 submit 화면에서 `03-covid19-katib-random.yaml` 내용을 copy & paste하고 submit 버튼 클릭